In [1]:
!nvidia-smi

Thu Jan  5 12:22:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.85.02    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:1A:00.0 Off |                  Off |
| 30%   30C    P8    21W / 300W |      2MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    Off  | 00000000:1B:00.0 Off |                  Off |
| 49%   

In [2]:
from bean.datasets import TextClassificationLoader
from bean.models import AutoModel
from bean.evaluate import TextClassificationEvaluate

/home/oogundep/.conda/envs/tevatron/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-05 12:22:32.089594: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-05 12:22:32.252915: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
swahili_news = TextClassificationLoader(dataset_name="swahili_news")
train_dataset, dev_dataset, test_dataset = swahili_news.load_data()

print(dev_dataset)
print(train_dataset)
print(test_dataset)

01/05/2023 12:22:36 - WARNING - datasets.builder - Found cached dataset swahili_news (/home/oogundep/.cache/huggingface/datasets/swahili_news/swahili_news/0.2.0/ed5c9a13b97e0d2864ff1e34bfbd38b2f2c54fea77acffcaef187eb4f13cf8cc)
01/05/2023 12:22:37 - WARNING - datasets.builder - Found cached dataset swahili_news (/home/oogundep/.cache/huggingface/datasets/swahili_news/swahili_news/0.2.0/ed5c9a13b97e0d2864ff1e34bfbd38b2f2c54fea77acffcaef187eb4f13cf8cc)


None
Dataset({
    features: ['text', 'label'],
    num_rows: 22207
})
Dataset({
    features: ['text', 'label'],
    num_rows: 7338
})


In [4]:
dataset_config = swahili_news.get_dataset_metadata(task="text_classification")
print(dataset_config)

{'name': 'swahili_news', 'languages_covered': ['swahili'], 'num_labels': 6, 'available_on_huggingface': True, 'dataset_link': 'https://huggingface.co/datasets/swahili_news', 'label2id': {'utumi': 0, 'kitaifa': 1, 'michezo': 2, 'kimataifa': 3, 'burudani': 4, 'afya': 5}, 'text_feature_name': 'text', 'label_feature_name': 'label', 'label_mapped': True}


In [5]:
model_class = AutoModel(model_name_or_path="Davlan/afro-xlmr-small", tokenizer_name="Davlan/afro-xlmr-small")
model, tokenizer = model_class.classification_model(num_labels=dataset_config['num_labels'], label2id=dataset_config['label2id'], seed=0)

Some weights of the model checkpoint at Davlan/afro-xlmr-small were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at Davlan/afro-xlmr-small and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classif

In [6]:
evaluation_configs={
    'max_length': 128,
    'truncation': True,
    'batch_size': 64,
    'padding': 'max_length',
    'pad_to_max_length': True,
    'output_dir': "models/afroxlmr-base",
    'label_to_id': dataset_config['label2id'],
    'text_feature_name': dataset_config['text_feature_name'],
    'label_feature_name': dataset_config['label_feature_name'],
    'label_mapped': dataset_config['label_mapped'],
}
evaluator = TextClassificationEvaluate(model=model,tokenizer=tokenizer, **evaluation_configs)

In [7]:
finetune_configs = {
    'num_train_epochs': 3,
    'learning_rate': 2e-5,
    'warmup_steps': 0,
    'weight_decay': 0.0,
    'gradient_accumulation_steps': 1,
    'adam_epsilon': 1e-8,
    'max_grad_norm': 1.0,
    'logging_steps': 100,
    'checkpointing_steps': '1000',
    'train_batch_size': 64,
    'lr_scheduler_type': 'linear',
    'num_warmup_steps': 0,
    'output_dir': "models/afroxlmr-base",
    'seed': 0,
    'with_tracking': True
}
scores = evaluator.finetune_evaluate(train_dataset=train_dataset, eval_dataset=test_dataset, config=finetune_configs)

01/05/2023 12:24:17 - WARNING - datasets.fingerprint - Parameter 'function'=<bound method TextClassificationEvaluate._tokenize_dataset of <bean.evaluate.evaluate_classifier.TextClassificationEvaluate object at 0x7f521516c610>> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
01/05/2023 12:24:17 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /home/oogundep/.cache/huggingface/datasets/swahili_news/swahili_news/0.2.0/ed5c9a13b97e0d2864ff1e34bfbd38b2f2c54fea77acffcaef187eb4f13cf8cc/cache-629f6fbed82c07cd.arrow
01/05/2023 12:24:20 - INFO - bean.evaluate.eval

In [8]:
print(scores)

{'accuracy': 0.9275006813845734, 'f1': 0.8715676859053234, 'precision': 0.877179924170688, 'recall': 0.866339612756185}
